In [1]:
import numpy as np
import chess
import math
import random
import numpy as np
import chess.polyglot

In [2]:
def play_n(BOARD,n, black, white):
     for k in range(n):
        if BOARD.turn:
            BOARD.push(best_move(b, white))
        else:
            BOARD.push(best_move(b, black))
        if BOARD.outcome() != None:
            return(board, eval_board(board))
    return(board, eval_board(board))

In [ ]:
#deep eval
def board_to_tensor(board):
    tensor = np.zeros((1,15, 8, 8), dtype=np.float32)
    values = [1, 2.6, 2.7, 4.1, 7.8, 15.4]
    piece_map = board.piece_map()
    prediction = game_stage(total_value(board))
    for square, piece in piece_map.items():
        piece_type = piece.piece_type
        color = (1 if piece.color else -1)
        piece_value = values[piece_type-1] * color
        layer = int(piece_type-1 + 3*(1+color))
        if piece_type == chess.PAWN:
            n = chess.square_rank(square)
            if piece.color == False:
                n = 7-n
            piece_value = pawn_value(n, prediction, (board, square, piece.color))
        if piece_type == chess.KING:
            n = chess.square_rank(square)
            if piece.color == False:
                n = 7-n
            piece_value += (chess.square_file(square), n, prediction)
        tensor[0,layer, chess.square_rank(square), chess.square_file(square)] = piece_value
        valid_squares = list(board.attacks(square))
        for s in valid_squares:
            tensor[0,layer, chess.square_rank(s), chess.square_file(s)] += 0.05*color
    turn = (1 if board.turn else -1)
    tensor[0, 12, 0, 0] = (0.25 if board.has_kingside_castling_rights(True) else 0)
    tensor[0, 12, -1, 0] = (-0.25 if board.has_kingside_castling_rights(False) else 0)
    tensor[0, 12, 0, -1] = (0.25 if board.has_queenside_castling_rights(True) else 0)
    tensor[0, 12, -1, -1] = (-0.25 if board.has_queenside_castling_rights(False) else 0)
    tensor[0, 13, :, :] = turn/640
    if board.is_checkmate():
        tensor[0, 14, :, :] = -turn
    return torch.from_numpy(tensor)

def fen_to_tensor(fen):
    board = chess.Board(fen)
    return(board_to_tensor(board))

In [ ]:
#position analysis
def is_passing_pawn(board, square, color):
    rank = chess.square_rank(square)
    file = chess.square_file(square)
    if file == 0:
         n_files = [0,1]
    if file == 7:
        n_files = [6,7]
    else:
        n_files = [file-1, file, file+1]
    if (color == True):
        for f in n_files:
            for r in range(rank+1, 8):
                square = rank * 8 + file
                if board.piece_at(target_square) == chess.Piece(chess.PAWN, not color):
                    return(False)
        return(True)
    else:
        for f in n_files:
            for r in range(0, rank):
                square = rank * 8 + file
                if board.piece_at(target_square) == chess.Piece(chess.PAWN, not color):
                    return(False)
        return(True)

    return False
def pawn_value(n, prediction, passing):
    value = np.array([[1,1,1,1.05,0.95,0.7,1.5,6], [1,1,1,1.05,1,0.8,1.8,6], [1,1,1.1,1.2,1.3,1.4,2.1,6], [1,1,1.2,1.4,1.6,1.9,2.8,6]])
    passing_value = np.array([[1,1,1,1,0.95,1.2,1.5,6], [1,1,1,1.05,1,0.8,1.8,6], [1,1,1.2,1.4,1.6,1.8,2.1,6], [1,1,1.3,1.6,1.9,2.3,2.8,6]])
    if passing:
        return np.sum(passing_value[:,n]*prediction)
    else:
        return np.sum(value[:,n]*prediction)
def king_value(file, n, prediction):
    file_value = np.array([[0.2,0.4,0.3,0,0,0.3,0.4,0.2],[0.2,0.8,0.8,0,0,0.8,0.8,0.2],[0,0.3,0.5,0.6,0.6,0.5,0.3,0],[0,0.3,0.5,0.5,0.5,0.5,0.3,0]])
    rank_value = np.array([[2, 1.7, 1.4, 1, 0.6, 0, 0,0],[2, 1.8, 1.5, 1.1, 0.6, 0, 0,0],[1.1, 1.3, 1.5, 1.7, 1.8, 1.6, 1.4, 1.2],[0.7, 1, 1.4, 1.7, 1.9, 2, 1.2,0.7]])
    return np.sum(file_value[:,file]*prediction + rank_value[:,n]*prediction)


In [ ]:
#eval
def eval_board(board):
    #will be replaced by a deep network
    #tensor = board_to_tensor(board)[0]
    score = 0
    if board.is_checkmate() and board.turn:
        return(-1000)
    if board.is_checkmate() and board.turn == False:
        return(1000)
    if board.is_stalemate():
        return(0)
    if board.is_insufficient_material():
        return(0)
    if board.has_insufficient_material(board.turn):
        return(-100)
    if board.has_insufficient_material(board.turn == False):
        return(100)
    values = [1, 2.6, 2.7, 4.1, 7.8, 15.4]
    piece_map = board.piece_map()
    prediction = game_stage(total_value(board))
    for square, piece in piece_map.items():
        piece_type = piece.piece_type
        color = (1 if piece.color else -1)
        piece_value = values[piece_type-1] * color
        if piece_type == chess.PAWN:
            n = chess.square_rank(square)
            if piece.color == False:
                n = 7-n
            piece_value = pawn_value(n, prediction, (board, square, piece.color))* color
        if piece_type == chess.KING:
            n = chess.square_rank(square)
            if piece.color == False:
                n = 7-n
            piece_value += king_value(chess.square_file(square), n, prediction) * color
        score += piece_value
        valid_squares = list(board.attacks(square))
        for s in valid_squares:
            score += 0.05*color
    turn = (1 if board.turn else -1)
    score += (0.25 if board.has_kingside_castling_rights(True) else 0)
    score += (-0.25 if board.has_kingside_castling_rights(False) else 0)
    score += (0.25 if board.has_queenside_castling_rights(True) else 0)
    score += (-0.25 if board.has_queenside_castling_rights(False) else 0)
    return int(score*100)/100


In [ ]:
class ChessValueNet(torch.nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.C1 = torch.nn.Conv2d(6, 6, 5, padding=0)
        self.C2 = torch.nn.Conv2d(6, 16, 3, padding=1)
        self.C3 = torch.nn.Conv2d(16, 128, 4, padding=0)
        self.F1 = torch.nn.Linear(128, 64)
        self.OUT = torch.nn.Linear(64, 1)
        self.LR = torch.nn.LeakyReLU(negative_slope=0.1)

    def forward(self, x):
        x = self.C1(x)
        x = self.LR(x)
        x = self.C2(x)
        x = self.LR(x)
        x = self.C3(x)
        x = self.LR(x)
        x = x.view(x.shape[0], -1)  # flatten the feature maps into a long vector
        x = self.F1(x)
        x = self.LR(x)
        x = self.OUT(x)
        return x
def mutation(model, epsilon):
    for param in model.parameters():
        if param.requires_grad:
            param.data *= (1.0 + epsilon * torch.randn(param.data.size()))

In [ ]:
def best_move(b, net):
    Moves = list(b.legal_moves)
    
    if len(Moves) ==0:
        return(chess.Move.null())
    Moves = random.sample(Moves, len(Moves))
    values = []
        for k in range(len(Moves)):
            copy = b.copy()
            copy.push(Moves[k])
            values.append(net(board_to_tensor(copy)))
        return(Moves[values.argmax())

In [ ]:
def clone(net, board, epsilon, N):
    nets = []
    for k in range(N):
        new_net = mutation(net.copy(), epsilon)
        nets.append([new_net, board.copy(), 0])
    return(nets)

In [ ]:
def train(net, epochs, N, epsilon, depth):
    board = chess.Board()
    nets = clone(net, board, epsilon, N)
    n_game = 0 
    
    for e in range(epochs):
        nets = random.sample(new_nets, N)
        black = nets.pop(0)
        for k in range(N):        
            nets[k][1], nets[k][2] = play_n(nets[k][1], depth, black, nets[k][0])
            #sort nets
        nets = nets[:3]
        if nets[0][1].outcome() == None: 
            board = nets[0][1]
        else:
            board = chess.Board()
            n_game +=1
            print(str(n_game) + " games played")
        new_nets = []
        length = [(N-1)//3, (N-1)//3, (N-1)-2*((N-1)//3)]
        for k in range(3):
            new_nets += clone(nets[k], board, epsilon, length[k])
        black[1] = board
        nets += black   
    return(nets[0][0], n_game*(N-1))
    
        
        

In [ ]:
net = 
epochs = 1000
N = 10
epsilon = 0.05
depth = 11

In [ ]:

best_net, n_game = train(net, epochs, N, epsilon, depth)

In [ ]:
def save(net,n_game):
    password = "train"
    if password !="train":
        print("wrong password")
        return(0)
    torch.save(net.state_dict(), "C:\Users\xiaji\école d'ing 2A\Chess AI\trained_net " + str(n_game))
    print('model saved')

In [ ]:
save(best_net)